<a href="https://colab.research.google.com/github/mrcreasey/oc-ds-p8-pyspark-deploy/blob/main/pyspark_sur_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pyspark sur Google Colab

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,521 kB]
Hit:14 http://pp

In [2]:

import os
import sys
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"



In [3]:
import pyspark
import findspark
findspark.init()
findspark.find()

'/content/spark-3.3.0-bin-hadoop3'

In [4]:
from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession.builder.appName("Our First Spark Example").getOrCreate()

spark

# Copy images

In [5]:
import io
from PIL import Image  # https://pillow.readthedocs.io/en/4.3.x/
import requests  # http://docs.python-requests.org/en/master/


# example image url: https://github.com/Horea94/Fruit-Images-Dataset/tree/master/Training/
def download_image(url, image_file_path):
    # https://gist.github.com/mjdietzx/545fa2874b2688e9bcb71e2ee92cd5a0
    r = requests.get(url, timeout=4.0)
    if r.status_code != requests.codes.ok:
        assert False, 'Status code error: {}.'.format(r.status_code)

    with Image.open(io.BytesIO(r.content)) as im:
        im.save(image_file_path)

    # print(f'Image downloaded from url: {url} and saved to: {image_file_path}')

if not os.path.exists('data'): os.makedirs('data')

# test
url='https://raw.githubusercontent.com/Horea94/Fruit-Images-Dataset/master/Training/Raspberry/0_100.jpg'
download_image(url,'data/raspberry.jpg')

In [6]:
import os
SRC= 'https://raw.githubusercontent.com/Horea94/Fruit-Images-Dataset/master/Training'
DEST = 'data/train'


FRUITS=['Blueberry','Kiwi','Orange','Pineapple','Raspberry']
ANGLES= [0,60,120,180,240,300]

def makedirs(path):
    if not os.path.exists(path):
        os.makedirs(path)

for fruit in FRUITS:
    src_path = f'{SRC}/{fruit}/'
    dest_path = f'{DEST}/{fruit}'
    makedirs(dest_path)
    for angle in ANGLES:
        filename =f'{angle}_100.jpg'
        try:
            download_image(f'{src_path}/{filename}',f'{dest_path}/{filename}')
        except:
            continue